In [2]:
using Pkg
Pkg.activate()
using SparseArrays, LinearAlgebra, Plots, LaTeXStrings
using KrylovKit: eigsolve

theme(:lime)

  Activating project at `C:\Users\saksh\.julia\environments\v1.7`


In [3]:
hilbert_dim(k, n) = binomial(k + n - 1, n - 1)

function generate_basis(N, L)
    if L > 1
        basis = zeros(Int16, (hilbert_dim(N, L), L))
        j = 1
    
        for n in 0:N
            d = hilbert_dim(n, L - 1)
            basis[j:(j + d - 1), 1] .= (N - n)
            basis[j:(j + d - 1), 2:end] = generate_basis(n, L - 1)
            j += d
        end
        
    else
        basis = [N]
    end

    return basis
end

generate_basis (generic function with 1 method)

In [11]:
function hop(state, i, j)
    res_state = copy(state)
    res_state[i] += 1
    res_state[j] -= 1
    return res_state
end

# tags to label the basis states
tag(state) = sum(sqrt.(100 .* (1:length(state)) .+ 3) .* state)
unzip(arr) = map(x -> getfield.(arr, x), fieldnames(eltype(arr)))

function generate_kinetic(t, basis)
    D, L = size(basis)
    basis_tags = tag.(eachslice(basis, dims = 1)) # generate tags for each basis
    inds, basis_tags = sortperm(basis_tags), sort(basis_tags) # sorting tag-list for efficient searching

    H_kin = Dict{Tuple{Int64, Int64}, Float64}() # store sparse values; (index) -> val

    for v in 1:D # iterate through basis vectors
        state = basis[v, :] # get vth basis state 

        for j in 1:(L-1) # iterate through states (hopping) 
            if state[j] > 0 
                i = mod1(j + 1, L) # Periodic BC
                u = inds[searchsortedfirst(basis_tags, tag(hop(state, i, j)))]
                H_kin[(u, v)] = get(H_kin, (u, v), 0.) - t * ((state[i] + 1) * state[j]) ^ 0.5
            
                i = mod1(j - 1, L) # Periodic BC
                u = inds[searchsortedfirst(basis_tags, tag(hop(state, i, j)))]
                H_kin[(u, v)] = get(H_kin, (u, v), 0.) - t * ((state[i] + 1) * state[j]) ^ 0.5
            end
        end
    end

    return sparse(unzip(keys(H_kin))..., collect(values(H_kin)))
end

generate_interaction(U, basis) = spdiagm(0.5 .* U .* sum(basis .* (basis .- 1), dims = 2) |> vec)

generate_number(mu, basis) = spdiagm(mu .* sum(basis, dims = 2) |> vec)

generate_hamiltonian(t, U, mu, basis) = generate_kinetic(t, basis) + generate_interaction(U, basis) - generate_number(mu, basis)

generate_hamiltonian (generic function with 1 method)

In [5]:
function a_lower(state, i)
    res_state = copy(state)
    res_state[i] -= 1
    return res_state
end

function a_raise(state, i)
    res_state = copy(state)
    res_state[i] += 1
    return res_state
end

function generate_a(basis, j)
    D, L = size(basis)
    basis_tags = tag.(eachslice(basis, dims = 1)) # generate tags for each basis
    inds, basis_tags = sortperm(basis_tags), sort(basis_tags) # sorting tag-list for efficient searching

    a_mat = Dict{Tuple{Int64, Int64}, Float64}() # store sparse values; (index) -> val

    for v in 1:D # iterate through basis vectors
        state = basis[v, :] # get vth basis state 

        if state[j] > 0 
            u = inds[searchsortedfirst(basis_tags, tag(a_lower(state, j)))]
            a_mat[(u, v)] = get(a_mat, (u, v), 0.) + sqrt(state[j])
        end
    end

    return sparse(unzip(keys(a_mat))..., collect(values(a_mat)))
end

function generate_a_dag(basis, j)
    D, L = size(basis)
    basis_tags = tag.(eachslice(basis, dims = 1)) # generate tags for each basis
    inds, basis_tags = sortperm(basis_tags), sort(basis_tags) # sorting tag-list for efficient searching

    a_mat = Dict{Tuple{Int64, Int64}, Float64}() # store sparse values; (index) -> val

    for v in 1:D # iterate through basis vectors
        state = basis[v, :] # get vth basis state 

        u = searchsortedfirst(basis_tags, tag(a_raise(state, j)))

        if(u < length(basis_tags) + 1) # raised state may be outside our truncated basis state
            u = inds[u]
            a_mat[(u, v)] = get(a_mat, (u, v), 0.) + sqrt(state[j] + 1)
        end
    end

    return sparse(unzip(keys(a_mat))..., collect(values(a_mat)))
end

generate_a_dag (generic function with 1 method)

In [37]:
generate_a(generate_basis(3, 3), 2)

3×9 SparseMatrixCSC{Float64, Int64} with 6 stored entries:
  ⋅   1.0   ⋅   1.41421  1.0   ⋅   1.73205   ⋅        ⋅ 
  ⋅    ⋅    ⋅    ⋅        ⋅    ⋅    ⋅       1.41421   ⋅ 
  ⋅    ⋅    ⋅    ⋅        ⋅    ⋅    ⋅        ⋅       1.0

In [6]:
function generate_fock_hamiltonian(t, U, mu, n_max, num_sites)
    basis = []
    hamiltonians = []
    for n in 1:n_max
        push!(basis, generate_basis(n, num_sites))
        push!(hamiltonians, generate_hamiltonian(t, U, mu, basis[n]))
    end

    return vcat(basis...), blockdiag(hamiltonians...)
end

generate_fock_hamiltonian (generic function with 1 method)

In [8]:
basis, H = generate_fock_hamiltonian(0.01, 1, 2, 3, 3);

In [9]:
H

19×19 SparseMatrixCSC{Float64, Int64} with 79 stored entries:
⠿⢇⣀⠀⠀⠀⠀⠀⠀⠀
⠀⠘⣟⣽⡂⠀⠀⠀⠀⠀
⠀⠀⠈⠈⢱⣶⢤⡀⠀⠀
⠀⠀⠀⠀⠀⠳⡻⢎⡳⠄
⠀⠀⠀⠀⠀⠀⠙⠎⠻⠆